# 0. Import Packages

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import os
import glob21
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import math

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras_preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Dropout
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import model_from_json
from keras import optimizers

# 1. Load data

In [72]:
# 온도 선택 [N10, 0, 10, 20, 25, 30, 40, 50]
# temp 변수에 설정
temp = 'N10'
mode = ['DST', 'FUDS' ,'US06']
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

for m in mode:
    data_dir = os.path.join(parent_dir, f'DB Preprocessing/refined_data/{temp}/{m}')
    file_names = os.listdir(data_dir)
    for file_name in file_names:
        csv_dir = os.path.join(data_dir, file_name)
        if '007' in file_name:
            num = '007'
        else: 
            num = '008'
        globals()['csv_{}'.format(f'{m}_{num}')] = pd.DataFrame(pd.read_csv(csv_dir))
        print(f'csv_{m}_{num} 생성 완료')
print(csv_DST_007)

csv_DST_007 생성 완료
csv_DST_008 생성 완료
csv_FUDS_007 생성 완료
csv_FUDS_008 생성 완료
csv_US06_007 생성 완료
csv_US06_008 생성 완료
      Current(A)  Voltage(V)  Temperature (C)_1        SoC
0       0.000191    3.526907          -8.332584  80.000000
1       0.000191    3.526599          -8.565621  80.000000
2       0.000191    3.526291          -8.332584  80.000000
3       0.000003    3.525676          -8.332584  80.000000
4       0.000378    3.525984          -8.332584  80.000000
...          ...         ...                ...        ...
5279   -0.480845    2.945211          -7.249700   0.356594
5280   -3.849219    2.086517          -7.094693   0.275559
5281   -3.849219    2.045275          -7.094693   0.176988
5282   -3.849219    2.017267          -6.985489   0.079046
5283   -3.849219    1.999724          -7.094693   0.000000

[5284 rows x 4 columns]


# 2. V, I, T / SoC 분리 

In [ ]:
num = ['007','008']
for m in mode:
    for n in num:
        var_name = f'csv_{m}_{n}'
        csv = globals()[var_name]
        globals()[f'input_{m}_{n}'] = csv[['Current(A)', 'Voltage(V)', 'Temperature (C)_1']]
        globals()[f'output_{m}_{n}'] = csv[['SoC']]
print(input_DST_007)
print(output_DST_007)
    